# Bitcoin derivation and transaction manual

In [1]:
from pybip44 import HDPrivateKey
import requests
import pandas as pd
from bitcoin_gp import BitcoinGP, Fee

from IPython.display import display
from ipywidgets import widgets, Layout

In [2]:
from halo import HaloNotebook as Halo

def waiting_animation(method):
    def spinner_animation(*args, **kw):
    
        spinner = Halo(text=f'Waiting for transaction...', text_color= 'cyan', color='green', spinner='dots')

        try:
            spinner.start()
            result = method(*args, **kw)
            spinner.stop_and_persist(text='Done!')
        except (KeyboardInterrupt, SystemExit):
            spinner.stop()
        
        return result    
    
    return spinner_animation

In [3]:
phrase = ''

text = widgets.Text(
    description='Phrase:',
    layout=Layout(width="70%")
)
display(text)

def handle(sender):
    global phrase
    phrase = text.value
    text.value =""
    
text.on_submit(handle)

## Derivation

In [4]:
master_key = HDPrivateKey.master_key_from_mnemonic(phrase, 'btc')
del phrase

# to switch between test and main network, you need to change 'coin' in derivation path m/44'/{coin}'/0
# for Main coin is 0
# for Test coin is 1
root_key = HDPrivateKey.from_path(master_key, "m/44'/1'/0'")

private_keys = []
addresses = []

for i in range(10):
    private_key = HDPrivateKey.from_path(root_key, '{change}/{index}'.format(change=0, index=i))
    private_keys.append(private_key.to_hex())
    public_key = private_key.public_key
#     print(f'\tPublic Key: {public_key.to_hex()}')
#     print(f'\tPrivate Key: {private_key.to_hex()}')
    
    # and here you need to change main_net parameter
    addresses.append(public_key.address(main_net=False))

    
df = pd.DataFrame(addresses, columns = ['Address']) 

df

,Address
0,mtQbvrrZd9kDLLNfJHaHHhu1t3jHx1c3Jh
1,mrG7g5qttebyfWar9MjYex9aDsDdVFGFLg
2,mwPdt1EndTm1C1CZyXNUjYmKACo5JTRonZ
3,mssTb1GWSotwnq4UVxhKashVhZLo1zLqEC
4,mr6fvKLjX2vsctMgirAMbSrYorPijUbpeK
5,mn6ooSwPCwR4NpW6Ce8TKvy85zVM8tcY7j
6,mnXM94JpsQjawV7b4m6WuZRj9dBijhamBg
7,msFoXfhbwMKsVzNsj39hZeVhrmWBQJQ5iK
8,mxVzrH4MSieXT4KbG53cQ4BmFw66kQuKnG
9,mmSzVX7xnRBNTDq9ahJzZZCL3zfanUizaU


## Balances

In [5]:
decimals = 10 ** 8

# here test net node on https://testgate.geniepay.io/v1/
# and main net on https://gate.geniepay.io/v1/
# url = 'https://gate.geniepay.io/v1/'
url = 'https://testgate.geniepay.io/v1/'
btc_gp = BitcoinGP(url, main_net = False)

balances_confirmed = []

balances_unconfirmed = []


for address in addresses:
    balance = btc_gp.get_balance(address)
    balances_confirmed.append(balance['confirmed'] / decimals)
    balances_unconfirmed.append(balance['unconfirmed'] / decimals)
    

df = pd.DataFrame(zip(addresses, balances_confirmed, balances_unconfirmed), columns = ['Address', 'Balance (confirmed)', 'Balance (unconfirmed)']) 

df

,Address,Balance (confirmed),Balance (unconfirmed)
0,mtQbvrrZd9kDLLNfJHaHHhu1t3jHx1c3Jh,0.240516,0.0
1,mrG7g5qttebyfWar9MjYex9aDsDdVFGFLg,0.063419,0.0
2,mwPdt1EndTm1C1CZyXNUjYmKACo5JTRonZ,0.024571,0.0
3,mssTb1GWSotwnq4UVxhKashVhZLo1zLqEC,0.002687,0.0
4,mr6fvKLjX2vsctMgirAMbSrYorPijUbpeK,0.000616,0.0
5,mn6ooSwPCwR4NpW6Ce8TKvy85zVM8tcY7j,0.000995,0.0
6,mnXM94JpsQjawV7b4m6WuZRj9dBijhamBg,0.000395,0.0
7,msFoXfhbwMKsVzNsj39hZeVhrmWBQJQ5iK,0.000686,0.0
8,mxVzrH4MSieXT4KbG53cQ4BmFw66kQuKnG,0.001601,0.0
9,mmSzVX7xnRBNTDq9ahJzZZCL3zfanUizaU,0.000048,0.0


In [6]:
@waiting_animation
def wait_for_transaction(transaction_hash):
    while True:
        transaction = btc_gp.get_transaction(transaction_hash)
        if transaction['height']:
            return transaction
        time.sleep(5)

## Transaction

In [8]:
# https://testgate.geniepay.io  - testnet
# https://gate.geniepay.io  - mainnet

account_from = [addresses[0], private_keys[0]]
account_to = [addresses[1], private_keys[1]]

signed_transaction = btc_gp.build_transaction(private_key=account_from[1],
                     address_from=account_from[0],
                     address_to=account_to[0],
                     amount=0.0001,
                     fee=Fee.FAST)

transaction_hash = btc_gp.send_transaction(signed_transaction)

print('\nTransaction hash:\n' + transaction_hash)
print('\nLink to BlockCypher if testnet:\n' + 'https://live.blockcypher.com/btc-testnet/tx/' + transaction_hash)
# print('\nLink to BlockCypher if mainnet:\n' + 'https://live.blockcypher.com/btc/tx/' + transaction_hash)


Transaction hash:
91c52fd537598e161dc4cc7d77d27a7f7c7a6830468136d514d8255df53136b1

Link to BlockCypher if testnet:
https://live.blockcypher.com/btc-testnet/tx/91c52fd537598e161dc4cc7d77d27a7f7c7a6830468136d514d8255df53136b1


In [9]:
wait_for_transaction(transaction_hash)

Output()

{'hash': '91c52fd537598e161dc4cc7d77d27a7f7c7a6830468136d514d8255df53136b1',
 'inputs': [{'address': 'mtQbvrrZd9kDLLNfJHaHHhu1t3jHx1c3Jh', 'value': 1},
  {'address': 'mtQbvrrZd9kDLLNfJHaHHhu1t3jHx1c3Jh', 'value': 18417220}],
 'outputs': [{'address': 'mrG7g5qttebyfWar9MjYex9aDsDdVFGFLg', 'value': 10000},
  {'address': 'mtQbvrrZd9kDLLNfJHaHHhu1t3jHx1c3Jh', 'value': 18406581}],
 'height': 1663267,
 'received': '2020-01-21T18:35:30Z'}

## Spend all assets

In [ ]:
account_from = [addresses[0], private_keys[0]]
account_to = [addresses[1], private_keys[1]]

signed_transaction_all_assets = btc_gp.build_transaction_all_assets_to_address(private_key=account_from[1],
                     address_from=account_from[0],
                     address_to=account_to[0],
                     fee=Fee.FAST)

transaction_all_assets_hash = btc_gp.send_transaction(signed_transaction_all_assets)

print('\nTransaction hash:\n' + transaction_all_money_hash)
# print('\nLink to BlockCypher if testnet:\n' + 'https://live.blockcypher.com/btc-testnet/tx/' + transaction_hash)
print('\nLink to BlockCypher if mainnet:\n' + 'https://live.blockcypher.com/btc/tx/' + transaction_all_money_hash)